In [7]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import json

#### [le monde]

In [2]:
url = 'https://www.lemonde.fr/actualite-en-continu/'

links = []

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

sections = soup.find_all('section')

for section in sections:
    if 'teaser' in section['class']:
        link = section.find('a')['href']

        if not link.startswith('http'):
            link = 'https://www.lemonde.fr' + link

        links.append(link)

for index, link in enumerate(links):
    print(f'[{index}] {link}')

[0] https://www.lemonde.fr/international/article/2024/10/24/turquie-au-lendemain-de-l-attentat-pres-d-ankara-le-gouvernement-confirme-que-les-auteurs-sont-des-membres-du-pkk_6359218_3210.html
[1] https://www.lemonde.fr/societe/article/2024/10/24/hugues-mulliez-condamne-a-six-ans-de-prison-avec-sursis-apres-un-accident-mortel-en-grece_6359217_3224.html
[2] https://www.lemonde.fr/planete/article/2024/10/24/climat-les-nations-unies-prevoient-un-rechauffement-de-3-1-c-a-la-fin-du-siecle-si-aucun-effort-supplementaire-n-est-fait_6359210_3244.html
[3] https://www.lemonde.fr/societe/article/2024/10/24/nourrisson-enleve-dans-une-maternite-si-ses-parents-m-entendent-santiago-doit-etre-immediatement-conduit-a-l-hopital-alerte-le-procureur-de-bobigny_6359211_3224.html
[4] https://www.lemonde.fr/economie/article/2024/10/24/amazon-s-investit-dans-la-formation-au-numerique-en-france_6359207_3234.html
[5] https://www.lemonde.fr/afrique/article/2024/10/24/hama-amadou-le-phenix-de-la-politique-nigerien

In [3]:
text = []

def crawl_lemonde(link, index):
    print(f"[{index}] {link}")

    article_response = requests.get(link, allow_redirects=False)
    article_soup = BeautifulSoup(article_response.text, 'html.parser')

    title_text = ""
    desc_text = ""
    paragraph_text = ""

    title = article_soup.find('h1', class_='article__title')
    if title:
        title_text = title.get_text(strip=True)
        print(f"제목: {title_text}")

    desc = article_soup.find('p', class_='article__desc')
    if desc:
        desc_text = desc.get_text(strip=True)
        desc_text = re.sub(r'\s+', ' ', desc_text)
        print(f"설명: {desc_text}")

    paragraphs = article_soup.find_all('p', class_='article__paragraph')
    for paragraph in paragraphs:
        paragraph_text += paragraph.get_text(strip=True) + ' '
    print(f"본문: {paragraph_text.strip()}")

    print("\n" + "=" * 40 + "\n")

    combined_text = f'{title_text}\n{desc_text}\n{paragraph_text.strip()}'
    current_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    return {
        "url": link,
        "date": current_date,
        "content": combined_text
    }

In [4]:
for index, link in enumerate(links):
    article_data = crawl_lemonde(link, index)
    text.append(article_data)

[0] https://www.lemonde.fr/international/article/2024/10/24/turquie-au-lendemain-de-l-attentat-pres-d-ankara-le-gouvernement-confirme-que-les-auteurs-sont-des-membres-du-pkk_6359218_3210.html
제목: Turquie : au lendemain de l’attentat près d’Ankara, le gouvernement confirme que les auteurs sont « des membres du PKK »
설명: En représailles à cet attentat, qui a fait cinq morts en plus des deux assaillants, l’armée turque mène depuis la nuit des raids qui ont visé « quarante-sept cibles [du PKK] », a annoncé le ministère de la défense.
본문: Le pouvoir turc a identifié, jeudi 24 octobre, les deux auteurs del’attentat contre le siège de Turkish Aerospace Industries (Tusas), l’une des plus importantes entreprises du pays dans les domaines de l’aérospatial et de la défense, mercredi près d’Ankara, certifiant qu’il s’agissait de combattants du Parti des travailleurs du Kurdistan (PKK). L’organisation armée, bête noire d’Ankara, est classée comme terroriste par la Turquie, les Etats-Unis et l’Union

In [5]:
text[0]

{'url': 'https://www.lemonde.fr/international/article/2024/10/24/turquie-au-lendemain-de-l-attentat-pres-d-ankara-le-gouvernement-confirme-que-les-auteurs-sont-des-membres-du-pkk_6359218_3210.html',
 'date': '2024-10-24 15:10:07',
 'content': 'Turquie\xa0: au lendemain de l’attentat près d’Ankara, le gouvernement confirme que les auteurs sont «\xa0des membres du PKK\xa0»\nEn représailles à cet attentat, qui a fait cinq morts en plus des deux assaillants, l’armée turque mène depuis la nuit des raids qui ont visé « quarante-sept cibles [du PKK] », a annoncé le ministère de la défense.\nLe pouvoir turc a identifié, jeudi 24\xa0octobre, les deux auteurs del’attentat contre le siège de Turkish Aerospace Industries (Tusas), l’une des plus importantes entreprises du pays dans les domaines de l’aérospatial et de la défense, mercredi près d’Ankara, certifiant qu’il s’agissait de combattants du Parti des travailleurs du Kurdistan (PKK). L’organisation armée, bête noire d’Ankara, est classée comm

In [8]:
file_path = 'lemonde-10-24.json'
with open(file_path, 'w', encoding='utf-8') as json_file:
    json.dump(text, json_file, ensure_ascii=False, indent=4)

print(f"데이터가 {file_path}에 저장되었습니다.")


데이터가 lemonde-10-24.json에 저장되었습니다.


#### [Doctissimo]

In [9]:
url = 'https://www.doctissimo.fr/'

links = []

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

spans = soup.find_all('span', class_='h5 card__body__title')

for span in spans:
    #print(span)
    atag=span.find('a')
    links.append(atag['href'])

for index, link in enumerate(links):
    print(f'[{index}] {link}')

[0] https://www.doctissimo.fr/sexualite/violences-sexuelles/nicolas-bedos-condamne-quelle-est-la-limite-entre-une-drague-lourde-et-une-agression-selon-notre-psy/dce553_ar.html
[1] https://www.doctissimo.fr/sante/seule-une-personne-sur-100-peut-voir-ces-couleurs-est-ce-votre-cas-faites-le-test/2ecda8_ar.html
[2] https://www.doctissimo.fr/nutrition/je-suis-nutritionniste-et-voici-les-aliments-que-je-privilegie-en-cas-de-coup-de-pompe/971584_ar.html
[3] https://www.doctissimo.fr/sante/cancer-du-sein-8-femmes-sur-10-ignorent-ce-geste-simple-qui-pourrait-tout-changer/0185b5_ar.html
[4] https://www.doctissimo.fr/sante/cholesterol/ce-signe-sur-vos-orteils-est-a-surveiller-il-peut-indiquer-un-fort-taux-de-cholesterol/b310c6_ar.html
[5] https://www.doctissimo.fr/psychologie/mieux-se-connaitre/bonheur/comment-garder-son-ame-denfant-au-quotidien-les-astuces-de-notre-psychologue/35161b_ar.html
[6] https://www.doctissimo.fr/sante/maladies/cancer/cancer-du-sein-parlons-prevention/japprends/preventio

In [12]:
# from math import comb
# text=[]

# def crawl_doctissimo(link, index):
#     print(f"[{index}] {link}")

#     article_response = requests.get(link, allow_redirects=False)
#     article_soup = BeautifulSoup(article_response.text, 'html.parser')

#     title_text = ""
#     desc_text = ""
#     paragraph_text = ""

#     title=article_soup.find('h1')
#     title_text=title.get_text(strip=True)
#     text.append(title_text)

#     desc=article_soup.find('div',class_='doc-my-4')
#     if desc==None: pass
#     else:
#         desc_text=desc.get_text(strip=True)
#         desc_text=re.sub(r'\s+', ' ', desc_text)
#         text.append(desc_text)

#     contents=article_soup.find('div',class_='doc-content doc-px-2 doc-px-md-0')
#     if contents==None: pass
#     else:
#         p_tags=contents.find_all('p',class_=False)

#         for p in p_tags:
#             paragraph_text=p.get_text(strip=True)
#             text.append(paragraph_text)

#     print(f"제목: {title_text}")
#     print(f"설명: {desc_text}")
#     print(f"본문: {paragraph_text}")

#     print("\n" + "="*40 + "\n")

#     combined_text = f'{title_text}\n{desc_text}\n{paragraph_text.strip()}'
#     current_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

#     return {
#         "url": link,
#         "date": current_date,
#         "content": combined_text
#     }


In [17]:
from math import comb

text=[]

def crawl_doctissimo(link, index):
    print(f"[{index}] {link}")

    article_response = requests.get(link, allow_redirects=False)
    article_soup = BeautifulSoup(article_response.text, 'html.parser')

    title_text = ""
    desc_text = ""
    p_text = ""

    title=article_soup.find('h1')
    title_text=title.get_text(strip=True)
    text.append(title_text)

    desc=article_soup.find('div',class_='doc-my-4')
    if desc==None: pass
    else:
        desc_text=desc.get_text(strip=True)
        desc_text=re.sub(r'\s+', ' ', desc_text)
        text.append(desc_text)

    contents=article_soup.find('div',class_='doc-content doc-px-2 doc-px-md-0')
    if contents==None: pass
    else:
        p_tags=contents.find_all('p',class_=False)

        for p in p_tags:
            p_text += p.get_text(strip=True) + '\n'


    print(f"제목: {title_text}")
    print(f"설명: {desc_text}")
    print(f"본문: {p_text}")

    print("\n" + "="*40 + "\n")

    combined_text = f'{title_text}\n{desc_text}\n{p_text.strip()}' # Use p_text here
    current_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    return {
        "url": link,
        "date": current_date,
        "content": combined_text
    }

In [18]:
for index,link in enumerate(links):
    article_data=crawl_doctissimo(link,index)
    text.append(article_data)

[0] https://www.doctissimo.fr/sexualite/violences-sexuelles/nicolas-bedos-condamne-quelle-est-la-limite-entre-une-drague-lourde-et-une-agression-selon-notre-psy/dce553_ar.html
제목: Nicolas Bedos condamné : quelle est la limite entre une drague lourde et une agression selon notre psy ?
설명: L’acteur et réalisateur Nicolas Bedos a été condamné hier à 12 mois de prison dont 6 fermes (avec bracelet électronique) pour deux faits d’agressions sexuelles en boîte de nuit en 2023. L’occasion de se poser la question : qu’est-ce qui distingue la drague lourde d’un fait répréhensible ?
본문: L’acteur et réalisateur Nicolas Bedos a été condamné hier à 12 mois de prison dont 6 fermes (avec bracelet électronique) pour deux faits d’agressions sexuelles en boîte de nuit en 2023. L’occasion de se poser la question : qu’est-ce qui distingue la drague lourde d’un fait répréhensible ?
En mai et juin 2023, l’acteur et réalisateur Nicolas Bedos s’est rendu coupable de deux faits d’agression sexuelle, selon un ju

In [ ]:
# text=[]

# for index, link in enumerate(links):
#     print(f"[{index}] {link}")

#     article_response = requests.get(link,allow_redirects = False)
#     article_soup = BeautifulSoup(article_response.text, 'html.parser')

#     title = article_soup.find('h1')
#     title_text = title.get_text(strip=True)
#     #print(title_text)
#     text.append(title_text)

#     desc = article_soup.find('div', class_='doc-my-4')
#     if desc==None: pass
#     else:
#         desc_text = desc.get_text(strip=True)
#         #print(desc_text)
#         desc_text = re.sub(r'\s+', ' ', desc_text)
#         text.append(desc_text)

#     contents=article_soup.find('div', class_='doc-content doc-px-2 doc-px-md-0')
#     if contents==None: pass
#     else:
#         p_tags=contents.find_all('p',class_=False)

#         for p in p_tags:
#             p_text=p.get_text(strip=True)
#             #print(p_text)
#             text.append(p_text)

#     print(f"제목: {title_text}")
#     print(f"설명: {desc_text}")
#     print(f"본문: {p_text}")

#     print("\n" + "="*40 + "\n")

[0] https://www.doctissimo.fr/nutrition/5-aliments-riches-en-calories-qui-sont-excellents-pour-votre-sante-selon-une-nutritionniste/8cafe0_ar.html
제목: 5 aliments riches en calories qui sont excellents pour votre santé, selon une nutritionniste
설명: On pense souvent que tous les aliments riches en calories sont néfastes pour la santé. Une grossière erreur, selon notre experte Alexandra Murcier, diététicienne-nutritionniste.
본문: Voici donc les 5 aliments caloriques à privilégier pour faire du bien à votre santé.


[1] https://www.doctissimo.fr/sante/le-depistage-du-cancer-du-poumon-est-il-realisable-en-vie-reelle-oui-selon-un-pneumo-oncologue/78f278_ar.html
제목: Le dépistage du cancer du poumon est-il réalisable en vie réelle ? Oui, selon un pneumo-oncologue
설명: Le dépistage du cancer du poumon sera-t-il instauré un jour en France ? Et si oui, est-il réalisable en conditions réelles ? Pour répondre à ces questions, Doctissimo a interrogé le Dr Olivier Leleu, pneumo-oncologue et auteur d’un

In [19]:
len(text)

267

In [20]:
text[-1]

{'url': 'https://www.doctissimo.fr/html/dossiers/maux-hiver/articles/10875-bronchite-aigue-infectieuse.htm',
 'date': '2024-10-24 15:27:16',
 'content': "Bronchite : causes, symptômes et traitement de la bronchite aiguë\nLa bronchite aiguë infectieuse vous donne mal à la gorge, une vilaine toux, de la fièvre, des courbatures ou encore des maux de tête ! Synonyme d'inflammation des bronches, cette maladie est pourtant banale et parfaitement bénigne. Inutile donc de s'affoler, une simple surveillance suffit généralement pour qu'elle disparaisse en quelques jours.\n"}

In [22]:
file_name='doctissimo-10-24.json'
with open(file_name,'w',encoding='utf-8') as json_file:
    json.dump(text,json_file,ensure_ascii=False,indent=4)

print(f"데이터가 {file_name}에 저장되었습니다.")

데이터가 doctissimo-10-24.json에 저장되었습니다.


#### [텍스트 전처리-lemonde]

In [ ]:
with open('lemonde-10-24.json','r') as f:
    lemonde=f.readlines()
    for l in lemonde:
        l=l.lower()                             #소문자로 바꾸기
        l=re.sub(r'[^a-zA-ZÀ-ÿ0-9\s]', '', l)   #특수문자 제거
        print(l)

nourrisson enlevé dans une maternité  la voiture des parents retrouvée à charleroi en belgique

santiago âgé de dixsept jours était suivi comme grand prématuré dans un hôpital daulnaysousbois près de paris ses parents âgés dune vingtaine dannées sont soupçonnés de lavoir enlevé

la piste dune fuite en belgique se confirme la voiture du couple ayantenlevé son nouveauné grand prématuré dans une maternitéde la région parisienne a été retrouvée à charleroi mercredi 23 octobre dans la matinée

le véhicule de marque audi dont la photo et la plaque dimmatriculation française figuraient dans lavis de recherche lancé mardi soir en belgique a été découvert vers 9 heures  a souligné un porteparole de la police de charleroi sans autres précisions ni le parquet de mons chargé des investigations en belgique ni celui de bobigny en france nont commenté linformation

santiago âgé de dixsept jours était suivi comme grand prématuré dans un hôpital daulnaysousbois seinesaintdenis près de paris il est acti

In [ ]:
#불용어 리스트
stopword='stopwords-fr.txt'

with open(stopword, 'r', encoding='utf-8') as f:
    stop_words = [line.strip() for line in f]
    print(stop_words)

['a', 'abord', 'absolument', 'afin', 'ah', 'ai', 'aie', 'aient', 'aies', 'ailleurs', 'ainsi', 'ait', 'allaient', 'allo', 'allons', 'allô', 'alors', 'anterieur', 'anterieure', 'anterieures', 'apres', 'après', 'as', 'assez', 'attendu', 'au', 'aucun', 'aucune', 'aucuns', 'aujourd', "aujourd'hui", 'aupres', 'auquel', 'aura', 'aurai', 'auraient', 'aurais', 'aurait', 'auras', 'aurez', 'auriez', 'aurions', 'aurons', 'auront', 'aussi', 'autant', 'autre', 'autrefois', 'autrement', 'autres', 'autrui', 'aux', 'auxquelles', 'auxquels', 'avaient', 'avais', 'avait', 'avant', 'avec', 'avez', 'aviez', 'avions', 'avoir', 'avons', 'ayant', 'ayez', 'ayons', 'b', 'bah', 'bas', 'basee', 'bat', 'beau', 'beaucoup', 'bien', 'bigre', 'bon', 'boum', 'bravo', 'brrr', 'c', 'car', 'ce', 'ceci', 'cela', 'celle', 'celle-ci', 'celle-là', 'celles', 'celles-ci', 'celles-là', 'celui', 'celui-ci', 'celui-là', 'celà', 'cent', 'cependant', 'certain', 'certaine', 'certaines', 'certains', 'certes', 'ces', 'cet', 'cette', 'ce

In [ ]:
len(stop_words)

691

In [ ]:
!pip install transformers


In [ ]:
!pip install sentencepiece

In [ ]:
from transformers import CamembertTokenizer, CamembertForSequenceClassification

c:\Users\MSI\anaconda3\envs\WEB_AI\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
tokenizer = CamembertTokenizer.from_pretrained('camembert/camembert-base')
model = CamembertForSequenceClassification.from_pretrained('camembert/camembert-base')

c:\Users\MSI\anaconda3\envs\WEB_AI\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\MSI\.cache\huggingface\hub\models--camembert--camembert-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert/

In [ ]:
train_tokens = [[token for token in tokenizer.morphs(text) if token not in stop_words] for text in trainDF['SentimentText']]
